In [ ]:
import numpy as np
import pandas as pd
 
import copy
from collections import Counter
import collections
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
%matplotlib inline
 
from nltk.util import ngrams
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.corpus import wordnet as wn
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
 
import nltk, re, string, collections
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
 
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse 
 
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')


import random
import token
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
clean_data = pd.read_csv('drive/My Drive/Tugas Akhir/review_data_Preprocessing.csv')
display(clean_data.head())

,Unnamed: 0,review,sentiment,unigram
0,0,"['one', 'of', 'the', 'other', 'reviewers', 'ha...",positive,"['one', 'reviewer', 'mention', 'watch', 'oz', ..."
1,1,"['a', 'wonderful', 'little', 'production', 'th...",positive,"['wonderful', 'little', 'production', 'filming..."
2,2,"['i', 'thought', 'this', 'was', 'a', 'wonderfu...",positive,"['think', 'wonderful', 'way', 'spend', 'time',..."
3,3,"['basically', 'there', 's', 'a', 'family', 'wh...",negative,"['basically', 'family', 'little', 'boy', 'jake..."
4,4,"['petter', 'mattei', 's', 'love', 'in', 'the',...",positive,"['petter', 'mattei', 'love', 'time', 'money', ..."


In [ ]:
X = clean_data['review']
y = clean_data.sentiment

In [ ]:
Encoder = LabelEncoder()
y = Encoder.fit_transform(y)

In [ ]:
# ---------------------- Get Sparse Matrix & Tags , Unigram, BIgram and Trigram ------------------
def get_sparse_n_tags(file_names, dataset_name):
    
    # load preprocessed dataset as dataframe
    #LABEL_DATA = pd.read_csv(dataset_name, encoding = "ISO-8859-1", usecols=["sentiment"])
    tags = clean_data.sentiment
    
    # load sparse matrix
    unigram = sparse.load_npz(file_names[0]).toarray()
    bigram = sparse.load_npz(file_names[1]).toarray()
  
    
    return [unigram, bigram, tags]

sparse_file_names = [
                    "tfidf_mat_unigram.npz",
                    "tfidf_mat_bigram.npz"]
preprocessed_dataset_name = y

tfidf_sparse_unigram, tfidf_sparse_bigram, tags = \
                        get_sparse_n_tags(sparse_file_names, preprocessed_dataset_name)

In [ ]:
tfidf_sparse_unigram.shape, tfidf_sparse_bigram.shape

((50000, 3000), (50000, 3000))

In [ ]:
# ---------------------------- SPLIT DATASET -----------------------------------------------
def split_dataset(sparse, tags, split_ratio):
    # split dataset 
    return train_test_split(sparse, tags, test_size=split_ratio, random_state=42) 


tfidf_sparse_unigram_train, tfidf_sparse_unigram_test, tags_train, tags_test = split_dataset(tfidf_sparse_unigram, tags, 0.3)
tfidf_sparse_bigram_train, tfidf_sparse_bigram_test, tags_train, tags_test = split_dataset(tfidf_sparse_bigram, tags, 0.3)

In [ ]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(tfidf_sparse_unigram_train,tags_train)  

#Predict y value for test dataset
y_pred = nb_tfidf.predict(tfidf_sparse_unigram_test)
y_prob = nb_tfidf.predict_proba(tfidf_sparse_unigram_test)
print(classification_report(tags_test, y_pred))
print('Confusion Matrix:\n',metrics.confusion_matrix(tags_test, y_pred))

avg_score = (cross_val_score(nb_tfidf, tfidf_sparse_unigram_test, tags_test, cv=5))
print(avg_score)
print("%0.3f accuracy with a standard deviation of %0.3f" % (avg_score.mean(), avg_score.std()))

              precision    recall  f1-score   support

    negative       0.84      0.86      0.85      7411
    positive       0.86      0.84      0.85      7589

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000

Confusion Matrix:
 [[6384 1027]
 [1182 6407]]
[0.85366667 0.84366667 0.85466667 0.85433333 0.85133333]
0.852 accuracy with a standard deviation of 0.004


In [ ]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(tfidf_sparse_bigram_train,tags_train)  

#Predict y value for test dataset
y_pred = nb_tfidf.predict(tfidf_sparse_bigram_test)
y_prob = nb_tfidf.predict_proba(tfidf_sparse_bigram_test)
print(classification_report(tags_test, y_pred))
print('Confusion Matrix:\n',metrics.confusion_matrix(tags_test, y_pred))

avg_score = (cross_val_score(nb_tfidf, tfidf_sparse_bigram_test, tags_test, cv=5))
print(avg_score)

print("%0.3f accuracy with a standard deviation of %0.3f" % (avg_score.mean(), avg_score.std()))

              precision    recall  f1-score   support

    negative       0.84      0.83      0.84      7411
    positive       0.84      0.85      0.84      7589

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000

Confusion Matrix:
 [[6152 1259]
 [1159 6430]]
[0.83466667 0.824      0.83366667 0.831      0.82733333]
0.830 accuracy with a standard deviation of 0.004


In [ ]:
svc = LinearSVC()
svc.fit(tfidf_sparse_unigram_train, tags_train)

y_true = tags_test
y_pred = svc.predict(tfidf_sparse_unigram_test)

print(classification_report(tags_test, y_pred))
print('Confusion Matrix:\n',metrics.confusion_matrix(tags_test, y_pred))
avg_score = (cross_val_score(svc, tfidf_sparse_unigram_test, tags_test, cv=5))
print(avg_score)

print("%0.3f accuracy with a standard deviation of %0.3f" % (avg_score.mean(), avg_score.std()))

              precision    recall  f1-score   support

    negative       0.89      0.88      0.89      7411
    positive       0.89      0.90      0.89      7589

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000

Confusion Matrix:
 [[6543  868]
 [ 791 6798]]
[0.87433333 0.873      0.87833333 0.873      0.87133333]
0.874 accuracy with a standard deviation of 0.002


In [ ]:
svc = LinearSVC()
svc.fit(tfidf_sparse_bigram_train, tags_train)

y_true = tags_test
y_pred = svc.predict(tfidf_sparse_bigram_test)

print(classification_report(tags_test, y_pred))
print('Confusion Matrix:\n',metrics.confusion_matrix(tags_test, y_pred))
avg_score = (cross_val_score(svc, tfidf_sparse_bigram_test, tags_test, cv=5))
print(avg_score)

print("%0.3f accuracy with a standard deviation of %0.3f" % (avg_score.mean(), avg_score.std()))

              precision    recall  f1-score   support

    negative       0.84      0.83      0.84      7411
    positive       0.84      0.85      0.84      7589

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000

Confusion Matrix:
 [[6176 1235]
 [1139 6450]]
[0.826      0.826      0.82766667 0.82066667 0.82533333]
0.825 accuracy with a standard deviation of 0.002
